In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [19]:
# 데이터 가져 온 다음, 형태 맞춤
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("x_train.shape : ",x_train.shape)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print("x_train.shape : ",x_train.shape)

x_train.shape :  (60000, 28, 28)
x_train.shape :  (60000, 28, 28, 1)


In [39]:
# 데이터를 섞고, 배치 사이즈를 만듬
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [46]:
# 모델 구축
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10, activation="softmax")
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel() # 인스턴스화

In [47]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() # 손실 함수
optimizer = tf.keras.optimizers.Adam() # 옵티마이저

In [48]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [49]:
@tf.function
def train_step(images, labels):
    # 순전파
    with tf.GradientTape() as tape:
        predictions = model(images) # 구축한 모델로 예측
        loss = loss_object(labels, predictions) # 실제 정답과 비교
    
    # 역전파
    gradients = tape.gradient(loss, model.trainable_variables) # 손실률에대한 기술기 계산 
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # 갱신

    # 데이터 담음
    train_loss(loss)
    train_accuracy(labels, predictions) 

In [50]:
@tf.function
def test_step(images, labels):
    predictions = model(images) # 학습한 모델에 대해서 예측
    t_loss = loss_object(labels, predictions) # 실제 정답과 비교

    #데이터 담음
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [51]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # 학습
    for images, labels in train_ds:
        train_step(images, labels)

    # test 데이터로 평가
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    # 결과 추출
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print(template.format(epoch+1,
                            train_loss.result(),
                             train_accuracy.result()*100,
                             test_loss.result(),
                             test_accuracy.result()*100))

에포크: 1, 손실: 0.13733439147472382, 정확도: 95.95000457763672, 테스트 손실: 0.05784233286976814, 테스트 정확도: 98.05999755859375
에포크: 2, 손실: 0.09053395688533783, 정확도: 97.27999877929688, 테스트 손실: 0.052520934492349625, 테스트 정확도: 98.27999877929688
에포크: 3, 손실: 0.06755201518535614, 정확도: 97.96277618408203, 테스트 손실: 0.052586629986763, 테스트 정확도: 98.34667205810547
에포크: 4, 손실: 0.05411575362086296, 정확도: 98.35916900634766, 테스트 손실: 0.05360586196184158, 테스트 정확도: 98.36750030517578
에포크: 5, 손실: 0.04505843296647072, 정확도: 98.62800598144531, 테스트 손실: 0.05452002212405205, 테스트 정확도: 98.40800476074219
